In [54]:
# -*- coding: utf-8 -*-


import random
import json
import time
from matplotlib import pyplot as plt
import math
from docplex.mp.model import Model

start = time.time()

# import the data

dataFile = open('data/data12.json')  
data = json.load(dataFile)
accesses = data['Accesses']
obsRequests = data['ObservationRequests']
satPasses = data['Passes']
nPasses = len(satPasses)

# Creation d'un modèle
model = Model(name='Satellite-passes')

# liste de variables

obs = model.binary_var_list(range(len(obsRequests)), name='obss')
pas = model.binary_var_list(range(len(satPasses)), name='pass')
acc = model.binary_var_list(range(len(accesses)), name='acc')

sat_start_eff = model.integer_var_list(range(len(satPasses)), name='sat_start_eff')
sat_end_eff = model.integer_var_list(range(len(satPasses)), name='sat_end_eff')

# contrainte de capacité
for p in range(len(satPasses)):
    for i in satPasses[p]['accessIds']:
        model.add_constraint(pas[p]>=acc[i])

        #chaque accès doit durer au minimum 30 sec
        model.add_constraint(accesses[i]['end']-accesses[i]['start'] >= 30*acc[i])
        #sat_start_eff doit être le minimum
        model.add_constraint(sat_start_eff[p]-accesses[i]['end']+30 <= 100000*(1-acc[i]))
        #sat_end_eff doit être le maximum
        model.add_constraint(accesses[i]['start']-sat_end_eff[p]+30 <= 100000*(1-acc[i]))

for o in range(len(obsRequests)):
    for i in obsRequests[o]['accessIds']:
        model.add_constraint(obs[o]>=acc[i])
    model.add_constraint(obs[o]<=model.sum(acc[j] for j in obsRequests[o]['accessIds']))
                                  
# fonction objectif
costPass_entire = [((satPasses[i]['end'] - satPasses[i]['start'])*data['passCostPerTimeUnit']+data['fixedPassCost'])for i in range(len(satPasses))]

cost = model.sum(pas[i]*data['fixedPassCost'] + acc[i]*(accesses[i]['end']-accesses[i]['start'])*data['passCostPerTimeUnit'] for i in range(len(satPasses)) )
cover = model.sum(obs[i] for i in range(len(obsRequests)))

alpha=10
model.maximize(alpha*cover/len(obs)-cost/sum(costPass_entire))

model.print_information()

# Résolution du problème
solution = model.solve(log_output=True)


# Affichage de la solution
if solution is None:
    print("NO solution")
else: 
    PassesP = []
    k=0
    cout=0
    
    #on renregistre les passes effectuées et les coûts fixes associés
    for i in range(len(pas)):
        if pas[i].solution_value >= (1-0.0001):
            PassesP.append(j)
            cout+=data['fixedPassCost']
            
    #on ajoute les coûts dûs à la durée d'observation
    for j in range(len(acc)):
        if acc[j].solution_value >= (1-0.0001):
            cout+=acc[j].solution_value*(accesses[j]['end']-accesses[j]['start'])*data['passCostPerTimeUnit']
         
    #on compte le nombre d'observation faites
    for l in range(len(obs)):
        if obs[l].solution_value >= (1-0.0001):
            k+=1
    print('================================')
    print("Nombre de passes:",len(PassesP))
    print("Passes prises:",PassesP)
    print("cout total:", cout)
    print('Observations prises: ', k)

Model: Satellite-passes
 - number of variables: 4003
   - binary=3865, integer=138, continuous=0
 - number of constraints: 18080
   - linear=18080
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
Found incumbent of value 0.000000 after 0.00 sec. (0.20 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 10757 rows and 141 columns.
Aggregator did 3 substitutions.
Reduced MIP has 7320 rows, 3859 columns, and 17983 nonzeros.
Reduced MIP has 3859 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (27.88 ticks)
Probing time = 0.01 sec. (3.30 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 7320 rows, 3859 columns, and 17983 nonzeros.
Reduced MIP has 3859 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (21.55 ticks)
Probing time = 0.